In [1]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Clases / Functions

In [2]:
def Load_Data(num):
    if (num == -1): # All data
        X_all = []
        y_all = []
        for i in range(8):
            file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
            data = h5py.File(file_path, 'r')
            X = np.copy(data['image'])
            y = np.copy(data['type'])
            X = X[:, 0:23, :]
            X_all.append(X)
            y = y[0,0:X.shape[0]:1]
            y_all.append(y)
        A, N, E, T = np.shape(X_all)
        X_all = np.reshape(X_all, (A*N, E, T))
        y_all = np.reshape(y_all, (-1))
        y_all = y_all - 769
        ## Remove NAN
        index_Nan = []
        for i in range(A*N):
            for j in range(E):
                if (any(np.isnan(X_all[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X_all = np.delete(X_all, index_Nan, axis=0)
        y_all = np.delete(y_all, index_Nan)
        return (X_all, y_all)
    else:
        file_path = './../project_datasets/A0' + str(num) + 'T_slice.mat'
        data = h5py.File(file_path, 'r')
        X = np.copy(data['image'])
        y = np.copy(data['type'])
        X = X[:, 0:23, :]
        y = y[0,0:X.shape[0]:1]
        y = y - 769
         ## Remove NAN
        N, E, T = np.shape(X)
        index_Nan = []
        for i in range(N):
            for j in range(E):
                if (any(np.isnan(X[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X = np.delete(X, index_Nan, axis=0)
        y = np.delete(y, index_Nan)
        return (X, y)

# Load Data

In [3]:
X, y = Load_Data(-1) # -1 to load all datas
N, E, T = np.shape(X)
print (np.shape(X))

(2280, 23, 1000)


# Make DataLoaders

In [4]:
bs_train = 200
bs_val = 100
bs_test = 100
data = data_utils.TensorDataset(torch.Tensor(X), torch.Tensor(y))
dset = {}
dataloaders = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-bs_val-bs_test, bs_val, bs_test])
dataloaders['train'] = data_utils.DataLoader(dset['train'], batch_size=bs_train, shuffle=True, num_workers=1)
dataloaders['val'] = data_utils.DataLoader(dset['val'], batch_size=bs_val, shuffle=True, num_workers=1)
dataloaders['test'] = data_utils.DataLoader(dset['test'], batch_size=bs_test, shuffle=True, num_workers=1)

# Define Model

In [6]:
class myGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myGRU, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.conv_temp = nn.Conv2d(1,40,tuple([1,25]))
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.gru1 = nn.GRU(input_dim, hidden_dim, num_layer)
        self.act1 = nn.Tanh()
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, num_layer)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x = x.permute(2,0,1)
        out_gru1, _ = self.gru1(x)
        out_act1 = self.act1(out_gru1)
        out_gru2, _ = self.gru2(out_act1)
        out_gru2 = out_gru2[-1, :, :] # taking the last time seq
        out = self.classifier(out_gru2)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

In [12]:
dtype = torch.cuda.FloatTensor
hidden_dim = 10
num_classes = 4
num_epoches = 100
model = myGRU(E, hidden_dim, 2, num_classes)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Conv Training

In [13]:
best_acc = 0.35

In [14]:
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train.cuda())
        # print (out)
        loss = loss_fn(out, y_train.long().cuda())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
    if (val_acc > best_acc):
        best_acc = val_acc
        torch.save(model, 'best_GRU.pt')

(0 batch) loss: 1.392268
(1 batch) loss: 1.394019
(2 batch) loss: 1.394917
(3 batch) loss: 1.397715
(4 batch) loss: 1.403159
(5 batch) loss: 1.381242
(6 batch) loss: 1.384384
(7 batch) loss: 1.385979
(8 batch) loss: 1.394333
(9 batch) loss: 1.386945
(10 batch) loss: 1.394715
(Epoch 1 / 100) train_acc: 0.253846; val_acc: 0.250000
(0 batch) loss: 1.384087
(1 batch) loss: 1.392471
(2 batch) loss: 1.394963
(3 batch) loss: 1.383678
(4 batch) loss: 1.385215
(5 batch) loss: 1.388480
(6 batch) loss: 1.384465
(7 batch) loss: 1.385950
(8 batch) loss: 1.385149
(9 batch) loss: 1.385232
(10 batch) loss: 1.390082
(Epoch 2 / 100) train_acc: 0.281731; val_acc: 0.210000
(0 batch) loss: 1.386912
(1 batch) loss: 1.393416
(2 batch) loss: 1.384865
(3 batch) loss: 1.381992
(4 batch) loss: 1.381143
(5 batch) loss: 1.387919
(6 batch) loss: 1.384235
(7 batch) loss: 1.388036
(8 batch) loss: 1.380148
(9 batch) loss: 1.386385
(10 batch) loss: 1.387213
(Epoch 3 / 100) train_acc: 0.271635; val_acc: 0.220000
(0 batc

(10 batch) loss: 1.369026
(Epoch 25 / 100) train_acc: 0.314904; val_acc: 0.270000
(0 batch) loss: 1.363875
(1 batch) loss: 1.347612
(2 batch) loss: 1.352383
(3 batch) loss: 1.349591
(4 batch) loss: 1.372367
(5 batch) loss: 1.379668
(6 batch) loss: 1.330815
(7 batch) loss: 1.353197
(8 batch) loss: 1.363769
(9 batch) loss: 1.374349
(10 batch) loss: 1.396287
(Epoch 26 / 100) train_acc: 0.321154; val_acc: 0.280000
(0 batch) loss: 1.349379
(1 batch) loss: 1.344474
(2 batch) loss: 1.357122
(3 batch) loss: 1.352660
(4 batch) loss: 1.348694
(5 batch) loss: 1.359403
(6 batch) loss: 1.379706
(7 batch) loss: 1.359775
(8 batch) loss: 1.369636
(9 batch) loss: 1.344795
(10 batch) loss: 1.383516
(Epoch 27 / 100) train_acc: 0.323077; val_acc: 0.240000
(0 batch) loss: 1.357555
(1 batch) loss: 1.368705
(2 batch) loss: 1.372128
(3 batch) loss: 1.337598
(4 batch) loss: 1.344031
(5 batch) loss: 1.350093
(6 batch) loss: 1.364514
(7 batch) loss: 1.337596
(8 batch) loss: 1.385490
(9 batch) loss: 1.357123
(10 

(6 batch) loss: 1.311922
(7 batch) loss: 1.282791
(8 batch) loss: 1.270195
(9 batch) loss: 1.314723
(10 batch) loss: 1.256629
(Epoch 50 / 100) train_acc: 0.384615; val_acc: 0.210000
(0 batch) loss: 1.327783
(1 batch) loss: 1.324927
(2 batch) loss: 1.281069
(3 batch) loss: 1.310171
(4 batch) loss: 1.265589
(5 batch) loss: 1.335870
(6 batch) loss: 1.245579
(7 batch) loss: 1.310366
(8 batch) loss: 1.333459
(9 batch) loss: 1.266120
(10 batch) loss: 1.346670
(Epoch 51 / 100) train_acc: 0.385577; val_acc: 0.250000
(0 batch) loss: 1.294472
(1 batch) loss: 1.281593
(2 batch) loss: 1.280810
(3 batch) loss: 1.288242
(4 batch) loss: 1.307063
(5 batch) loss: 1.321187
(6 batch) loss: 1.298756
(7 batch) loss: 1.316409
(8 batch) loss: 1.307777
(9 batch) loss: 1.310876
(10 batch) loss: 1.308212
(Epoch 52 / 100) train_acc: 0.391827; val_acc: 0.230000
(0 batch) loss: 1.289772
(1 batch) loss: 1.253024
(2 batch) loss: 1.320425
(3 batch) loss: 1.292449
(4 batch) loss: 1.287926
(5 batch) loss: 1.298240
(6 b

Process Process-282:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 50, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


(4 batch) loss: 1.263684
(5 batch) loss: 1.235944


Exception ignored in: <bound method DataLoaderIter.__del__ of <torch.utils.data.dataloader.DataLoaderIter object at 0x7faff7ae4b70>>
Traceback (most recent call last):
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 333, in __del__
    self._shutdown_workers()
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 319, in _shutdown_workers
    self.data_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 345, in get
    return ForkingPickler.loads(res)
  File "/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/torch/multiprocessing/reductions.py", line 70, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.5/multiprocessing/resource_sharer.py", l

KeyboardInterrupt: 

# Best Model Test

In [10]:
best_model = torch.load('best_GRU.pt')
print (best_model.check_accuracy(dataloaders['val']))

0.35


/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/ipykernel_launcher.py:17: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
